# **Несколько примеров работы с запросами SQL**

Составьте таблицу из полей:  
name_of_fund — название фонда;  
name_of_company — название компании;  
amount — сумма инвестиций, которую привлекла компания в раунде.  
В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.

In [ ]:
SELECT f.name AS name_of_fund,
       st.name AS name_of_company,
       fr.raised_amount AS amount
FROM investment AS i
LEFT JOIN fund AS f ON f.id = i.fund_id
LEFT JOIN company AS st ON st.id = i.company_id
INNER JOIN 
           (SELECT *
            FROM funding_round 
            WHERE funded_at BETWEEN '2012-01-01' AND '2013-12-31') AS fr
ON fr.id = i.funding_round_id 
WHERE st.milestones > 6;

Выгрузите таблицу, в которой будут такие поля:  
название компании-покупателя;  
1.сумма сделки;  
2.название компании, которую купили;  
3.сумма инвестиций, вложенных в купленную компанию;  
4.доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.  

Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы.  
Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Ограничьте таблицу первыми десятью записями.

In [ ]:
SELECT c.name AS ascuiring_company,
       ac.price_amount,
       c_1.name AS ascuired_company,
       c_1.funding_total,
       ROUND(ac.price_amount/c_1.funding_total) AS percent
FROM acquisition AS ac
LEFT JOIN company AS c ON ac.acquiring_company_id = c.id
LEFT JOIN company AS c_1 ON ac.acquired_company_id = c_1.id
WHERE ac.price_amount > 0 AND c_1.funding_total > 0
ORDER BY ac.price_amount DESC, ascuired_company
LIMIT 10;

Выгрузите таблицу, в которую войдут названия компаний из категории social, получившие финансирование с 2010 по 2013 год включительно. Проверьте, что сумма инвестиций не равна нулю. Выведите также номер месяца, в котором проходил раунд финансирования.

In [ ]:
SELECT c.name, 
       EXTRACT(MONTH FROM CAST(f.funded_at AS date)) AS month
FROM company AS c
LEFT JOIN funding_round AS f ON c.id = f.company_id
WHERE f.funded_at BETWEEN '2010-01-01' AND '2013-12-31' 
      AND c.category_code = 'social'
      AND f.raised_amount > 0;

Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:  
1.номер месяца, в котором проходили раунды;  
2.количество уникальных названий фондов из США, которые инвестировали в этом месяце;  
3.количество компаний, купленных за этот месяц;  
общая сумма сделок по покупкам в этом месяце.

In [ ]:
WITH 
data_month AS (SELECT EXTRACT(MONTH FROM CAST(fr.funded_at AS date)) AS month_1,
               COUNT(DISTINCT f.name) AS amount_company_1
               FROM funding_round AS fr
               JOIN investment AS i ON fr.id = i.funding_round_id
               JOIN fund AS f ON i.fund_id = f.id
               WHERE EXTRACT(YEAR FROM CAST(fr.funded_at AS date)) BETWEEN '2010' AND '2013'
                  AND f.country_code = 'USA'
               GROUP BY month_1),
               
info_acquisition AS (SELECT EXTRACT(MONTH FROM CAST(a.acquired_at AS date)) AS month_1,
                     COUNT(a.acquired_company_id) AS amount_buy_company_1,
                     SUM(a.price_amount) AS sum_price_1
                     FROM acquisition AS a 
                     WHERE EXTRACT(YEAR FROM CAST(a.acquired_at AS date)) BETWEEN '2010' AND '2013'
                     GROUP BY month_1)

SELECT data_month.month_1 AS month,
       data_month.amount_company_1 AS amount_company,
       info_acquisition.amount_buy_company_1 AS amount_buy_company,
       info_acquisition.sum_price_1 AS sum_price
FROM data_month LEFT JOIN info_acquisition ON data_month.month_1 = info_acquisition.month_1;

Составьте сводную таблицу и выведите среднюю сумму инвестиций для стран, в которых есть стартапы, зарегистрированные в 2011, 2012 и 2013 годах. Данные за каждый год должны быть в отдельном поле. Отсортируйте таблицу по среднему значению инвестиций за 2011 год от большего к меньшему.

In [ ]:
WITH 
one AS (SELECT country_code AS country,
            AVG(funding_total) AS year_2011
        FROM company
        WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = 2011
        GROUP BY country),

two AS (SELECT country_code AS country,
            AVG(funding_total) AS year_2012
        FROM company
        WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = 2012
        GROUP BY country),
       
three AS (SELECT country_code AS country,
            AVG(funding_total) AS year_2013
        FROM company
        WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) = 2013
        GROUP BY country)        

SELECT one.country,
       one.year_2011,
       two.year_2012,
       three.year_2013
FROM one INNER JOIN two ON one.country = two.country
INNER JOIN three ON two.country = three.country
ORDER BY year_2011 DESC;